In [1]:
from src.code.audio import AUDIO_DATA
#from src.audio import AUDIO_DATA
edf_path = 'C:\DeepRESTORE\EEGAnotator\Data\F10\PictureNaming\eeg\\f10.edf'
xdf_path = 'C:\DeepRESTORE\\25-05-2024\EEG_AUDIO_Anotator\F10\PictureNaming\\xdf\\f10.xdf'
#edf_path = '/home/owais/GitHub/21-05-2024/EEGAnotator/F10/PictureNaming/eeg/sample.edf'
#xdf_path = '/home/owais/GitHub/21-05-2024/EEGAnotator/F10/PictureNaming/xdf/sample.xdf'
audio = AUDIO_DATA(filepath_xdf=xdf_path)


***************************Loading Audio data***************************
*********************************************************************************
***************************Loading .xdf file***************************
*******************************Completed*******************************
***************************Preprocessing Audio data***************************
***************************Started Bundling markers and audio timestamps***************************


In [2]:
from src.code.eeg import EEG_DATA

eeg = EEG_DATA(edf_path)

*********************************************************************************
***************************Loading .edf file***************************
C:\DeepRESTORE\EEGAnotator\Data\F10\PictureNaming\eeg\f10.edf
Extracting EDF parameters from C:\DeepRESTORE\EEGAnotator\Data\F10\PictureNaming\eeg\f10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
*******************************Completed*******************************
Checking for Interruptions
No interruptions detected.


In [5]:
audio.MARKERS_WORDS_TIMESTAMPS_AUDIO_STARTINDEX[:4]

[['ExperimentStarted', '', 1656407619.154994, 790877],
 ['StartBlockSaying', '', 1656407619.1551409, 790883],
 ['StartReading', 'TENEDOR', 1656407620.8462954, 865464],
 ['EndReading', 'TENEDOR', 1656407621.5558956, 896758]]

In [6]:
eeg.EVENTS[:4]

[['ExperimentEnded', 0.0, 9.0546875, 0, 4636, 4636],
 ['StartBlockSaying', 9.0546875, 11.021484375, 4636, 5643, 1007],
 ['StartReading', 11.021484375, 12.5234375, 5643, 6412, 769],
 ['EndReading', 12.5234375, 14.076171875, 6412, 7207, 795]]

In [7]:
eeg.TIME_STAMPS

array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       5.96849414e+03, 5.96849609e+03, 5.96849805e+03])

In [9]:
import numpy as np

def bundle_audio_markers_with_timestamps(markers, marker_timestamps, audio_timestamps):
    """
    Bundles audio markers with corresponding timestamps.

    Parameters:
        - markers (list): List of marker data.
        - marker_timestamps (list): List of timestamps corresponding to the markers.
        - audio_timestamps (np.ndarray): Array of timestamps corresponding to the audio data.

    Returns:
        - marker_word_timestamp (list): List of lists containing marker action, marker word, timestamp, and audio start index.
    """
    print('***************************Started Bundling markers and audio timestamps***************************')

    marker_word_timestamp = []

    # Pre-calculate the closest audio start indices for each marker timestamp
    audio_indices = np.searchsorted(audio_timestamps, marker_timestamps)
    audio_indices = np.clip(audio_indices, 1, len(audio_timestamps) - 1)
    left_distances = abs(audio_timestamps[audio_indices - 1] - marker_timestamps)
    right_distances = abs(audio_timestamps[audio_indices] - marker_timestamps)
    closest_indices = np.where(left_distances <= right_distances, audio_indices - 1, audio_indices)

    # Process each marker and its corresponding timestamp
    for index in range(len(markers)):
        marker_values = markers[index][0].split(':')

        marker_action = marker_values[0]
        marker_word = marker_values[1] if len(marker_values) > 1 else ''
        timestamp = marker_timestamps[index]
        audio_start_index = closest_indices[index]

        marker_word_timestamp.append([marker_action, marker_word, timestamp, audio_start_index])

    return marker_word_timestamp


In [10]:
x = bundle_audio_markers_with_timestamps(audio.MARKERS, audio.MARKERS_TIME_STAMPS, audio.AUDIO_TIME_STAMPS )

***************************Started Bundling markers and audio timestamps***************************


In [1]:
x

NameError: name 'x' is not defined

In [16]:
eeg.EVENTS[:4]

[['ExperimentEnded', 0.0, 9.0546875, 0, 4636, 4636],
 ['StartBlockSaying', 9.0546875, 11.021484375, 4636, 5643, 1007],
 ['StartReading', 11.021484375, 12.5234375, 5643, 6412, 769],
 ['EndReading', 12.5234375, 14.076171875, 6412, 7207, 795]]

In [13]:
y = audio.MARKERS_WORDS_TIMESTAMPS_AUDIO_STARTINDEX

In [14]:
y == x

True

In [5]:
audio_ts = audio.STREAMS[1]['time_stamps']
audio_ts[:4]

array([1.6564076e+09, 1.6564076e+09, 1.6564076e+09, 1.6564076e+09])

In [9]:
import numpy as np

index = np.argmin(abs(audio_ts - marker_ts[3]))
index

896758

In [12]:
audio_ts[index]

1656407621.5561934

In [13]:
audio.AUDIO_START_TIME

numpy.datetime64('2022-06-28T11:13:21.221')